In [1]:
import numpy as np
import pandas as pd
import os
import numpy as np
from pmdarima.arima import auto_arima
import pmdarima as pmd
import warnings
warnings.filterwarnings('ignore')
import mysql.connector
import findspark

import pyspark
from pyspark.sql import SparkSession
import json
from pyspark.sql.functions import col, lit, udf,trunc,concat_ws
from pyspark.sql.types import IntegerType,BooleanType,DateType
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display
from IPython.core.display import HTML, display
from builtins import max
from builtins import min
from tqdm import tqdm
import numpy as np
import pandas as pd
import os
import time
import numpy as np
from pmdarima.arima import auto_arima
import pmdarima as pmd
import warnings
warnings.filterwarnings('ignore')
import mysql.connector
import findspark
import pyspark
from pyspark.sql import SparkSession
import json
from pyspark.sql.functions import col, lit, udf,trunc,concat_ws
from pyspark.sql.types import IntegerType,BooleanType,DateType
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display
from IPython.core.display import HTML, display
from builtins import max
from builtins import min
import mysql.connector
import pandas as pd
pd.options.mode.chained_assignment = None  # Suppress the warning
from tqdm import tqdm
import numpy as np
import numpy as np
import pandas as pd
from tqdm import tqdm
from IPython.display import display
from IPython.core.display import HTML, display
from datetime import timedelta
import pandas as pd
from datetime import timedelta
import numpy as np
import openpyxl
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from openpyxl import load_workbook

In [2]:
pwd

'/home/VECVNET/zzasom/jupyter/01_Developement_work/01_Codes/Black Forecasting/Monte Carlo/Dec/1153/Builds'

In [3]:
# reading the path file which contains the path for credential json file
filepath = open("/home/VECVNET/CONFIG/path.json", "r")
path = json.load(filepath)
filepath.close()
path = list(path.values())[0]

In [4]:
%store -r enteredCred
user=enteredCred[0]
password=enteredCred[1]
db=enteredCred[2]
url=enteredCred[3]
driver=enteredCred[4]

In [5]:
with open(path + '/config.json') as f:
    data = json.load(f)
    growthRate = data['growthRate']
    cores_max = data['cores_max']
    executor_memory = data['executor_memory']
    driver_memory = data['driver_memory']
    debug_maxToStringFields = data['debug_maxToStringFields']
    default_parallelism = data['default_parallelism']
    driver_maxResultSize = data['driver_maxResultSize']
    spark_network_timeout = data['spark_network_timeout']

In [6]:
# initializing spark session
findspark.init()

In [7]:
# setting up the different config required in the server
# This configs help to run the spark session uninteruptedlly
# The config in the spark session can be readilly updated to improve the running process of the code
# The session would bind a port in spark UI and the progress and also be monitored.
# set warning as false to remove the messages

spark = SparkSession.builder.master("spark://10.106.111.117:7077") \
        .appName("Monte Carlo - purchasing group buyer counts") \
        .config("spark.cores.max",2) \
        .config("spark.driver.memory",driver_memory) \
        .config("spark.executor.memory", '20g') \
        .config("spark.debug.maxToStringFields",debug_maxToStringFields) \
        .config("spark.default.parallelism",default_parallelism) \
        .config("spark.driver.maxResultSize",driver_maxResultSize) \
        .config("spark.network.timeout",spark_network_timeout) \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/05/30 10:51:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
# importing arrow from spark for faster execution
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

#PURCHASUNG GROUP FILE

In [ ]:
# read the PG file

#/home/VECVNET/zzasom/jupyter/01_Developement_work/01_Codes/Black Forecasting/Monte Carlo/Dec/1153/Input

folder_path1 = "/home/VECVNET/zzasom/jupyter/01_Development_work/Black_Forecasting/Deployment/Input"

# Get yesterday's date in YYYYMMDD or any required format
#yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y%m%d')

# Build the expected filename
filename1 = f"MRPcontrollerPurchasingGrp.csv"  # Adjust pattern as per your naming scheme

# Full path to the file
file_path1 = os.path.join(folder_path1, filename1)

# Read the file if it exists
if os.path.exists(file_path1):
    df_PG = pd.read_csv(file_path1)
    print(df_PG.head(2))
else:
    print(f"No file found")

#df_PG['Material_PG_Concat'] = df_PG['Material'].astype(str) + "_" + df_PG['Purchasing Group Details'].astype(str)
df_PG.shape

In [ ]:
#df_PG['Material_PG_Concat'] = df_PG['Material'].astype(str) + "_" + df_PG['Purchasing Group Details'].astype(str)
df_gp = df_PG[['Material', 'Purchasing Group Details']]
df_gp['Material_PG_Concat'] = df_gp['Material'].astype(str) + "_" + df_gp['Purchasing Group Details'].astype(str)
print(df_gp.shape)

In [ ]:
df_gp.head(7)

#week - 01

# 11-5-2025 file

In [ ]:
# Set parameters
num_simulations = 10000  # Adjust as needed

# Generate yesterday's date in 'YYYYMMDD' format
yesterday = datetime.now() - timedelta(days=5)
date_str = yesterday.strftime('%Y%m%d')

# Construct filename dynamically
filename = f"MC_day_wise_{num_simulations}_{date_str}.xlsx"

# Define folder path and file path
folder_path = "/home/VECVNET/zzasom/jupyter/01_Development_work/Black_Forecasting/Deployment/Output"
file_path = os.path.join(folder_path, filename)

# Check and read the file
if os.path.exists(file_path):
    xls = pd.ExcelFile(file_path)
    for sheet_name in xls.sheet_names:
        df_MC11 = pd.read_excel(xls, sheet_name=sheet_name)
        if not df_MC11.empty:
            print(f"Reading data from sheet: {sheet_name}")
            #print(df_MC6.head(3))
            break
    else:
        print("All sheets are empty in the Excel file.")
else:
    print(f"No file found for date {date_str}: {filename}")


# select particular columns in PG data 
df1 = df_PG[['Material', 'Purchasing Group Details', 'MRP controller details']]
df1.rename(columns={'Material': 'Material Code'}, inplace=True)


merged_df11 = pd.merge(df_MC11, df1, on='Material Code', how='inner')
print(merged_df11.shape)

# Create concatenated column
merged_df11['Material_PG_Concat'] = merged_df11['Material Code'].astype(str) + "_" + merged_df11['Purchasing Group Details'].astype(str)


# Ensure the column is numeric (in case it's read as string)
merged_df11['Stockout Probability (%)'] = pd.to_numeric(merged_df11['Stockout Probability (%)'], errors='coerce')

# Filter rows with stockout probability >= 50%
high_risk_df11 = merged_df11[merged_df11['Stockout Probability (%)'] >= 50]

# Show the filtered DataFrame
print(high_risk_df11.shape)

#print(high_risk_df.head(2))

filtered_df11 = high_risk_df11[high_risk_df11['GIT'] == 0]
print(filtered_df11.shape)

filtered_df11 = filtered_df11[['Running Date','Material Code', 'GIT', 'Purchasing Group Details','Material_PG_Concat']]
filtered_df11.head(7)



In [ ]:
merged_dfgp_filtered_df11 = pd.merge(
    filtered_df11,
    df_gp[['Material_PG_Concat']],  # Only use the join column from df_gp
    on='Material_PG_Concat',
    how='inner'
)

print(merged_dfgp_filtered_df11.shape)
merged_dfgp_filtered_df11.head(7)

In [ ]:
pg_count_11may = (
    merged_dfgp_filtered_df11.groupby('Purchasing Group Details')
    .size()
    .reset_index(name='Buyer_counts_11May')
    .sort_values(by=['Purchasing Group Details', 'Buyer_counts_11May'], ascending=[True, False])
)

print(pg_count_11may.shape)
pg_count_11may.head(4)

# 12-05-2025 file

In [ ]:
#import os
#from datetime import datetime, timedelta
#import pandas as pd

# Set parameters
num_simulations = 10000  # Adjust as needed

# Generate yesterday's date in 'YYYYMMDD' format
yesterday = datetime.now() - timedelta(days=4)
date_str = yesterday.strftime('%Y%m%d')

# Construct filename dynamically
filename = f"MC_day_wise_{num_simulations}_{date_str}.xlsx"

# Define folder path and file path
folder_path = "/home/VECVNET/zzasom/jupyter/01_Development_work/Black_Forecasting/Deployment/Output"
file_path = os.path.join(folder_path, filename)

# Check and read the file
if os.path.exists(file_path):
    xls = pd.ExcelFile(file_path)
    for sheet_name in xls.sheet_names:
        df_MC12 = pd.read_excel(xls, sheet_name=sheet_name)
        if not df_MC12.empty:
            print(f"Reading data from sheet: {sheet_name}")
            #print(df_MC6.head(3))
            break
    else:
        print("All sheets are empty in the Excel file.")
else:
    print(f"No file found for date {date_str}: {filename}")


# select particular columns in PG data 
df12 = df_PG[['Material', 'Purchasing Group Details', 'MRP controller details']]
df12.rename(columns={'Material': 'Material Code'}, inplace=True)


merged_df12 = pd.merge(df_MC12, df12, on='Material Code', how='inner')
print(merged_df12.shape)

# Create concatenated column
merged_df12['Material_PG_Concat'] = merged_df12['Material Code'].astype(str) + "_" + merged_df12['Purchasing Group Details'].astype(str)


# Ensure the column is numeric (in case it's read as string)
merged_df12['Stockout Probability (%)'] = pd.to_numeric(merged_df12['Stockout Probability (%)'], errors='coerce')

# Filter rows with stockout probability >= 50%
high_risk_df12 = merged_df12[merged_df12['Stockout Probability (%)'] >= 50]

# Show the filtered DataFrame
print(high_risk_df12.shape)

#print(high_risk_df.head(2))

filtered_df12 = high_risk_df12[high_risk_df12['GIT'] == 0]
print(filtered_df12.shape)

filtered_df12 = filtered_df12[['Running Date','Material Code', 'GIT', 'Purchasing Group Details','Material_PG_Concat']]
filtered_df12.head(7)

In [ ]:
filtered_df11_filtered_df12 = pd.merge(
    filtered_df12,
    merged_dfgp_filtered_df11[['Material_PG_Concat']],  # Only use the join column from df_gp
    on='Material_PG_Concat',
    how='inner'
)

print(filtered_df11_filtered_df12.shape)
filtered_df11_filtered_df12.head(7)

In [ ]:
pg_count_12may = (
    filtered_df11_filtered_df12.groupby('Purchasing Group Details')
    .size()
    .reset_index(name='Buyer_counts_12May')
    .sort_values(by=['Purchasing Group Details', 'Buyer_counts_12May'], ascending=[True, False])
)

print(pg_count_12may.shape)
pg_count_12may.head(5)

# 13-05-2025 file

In [ ]:
#import os
#from datetime import datetime, timedelta
#import pandas as pd

# Set parameters
num_simulations = 10000  # Adjust as needed

# Generate yesterday's date in 'YYYYMMDD' format
yesterday = datetime.now() - timedelta(days=3)
date_str = yesterday.strftime('%Y%m%d')

# Construct filename dynamically
filename = f"MC_day_wise_{num_simulations}_{date_str}.xlsx"

# Define folder path and file path
folder_path = "/home/VECVNET/zzasom/jupyter/01_Development_work/Black_Forecasting/Deployment/Output"
file_path = os.path.join(folder_path, filename)

# Check and read the file
if os.path.exists(file_path):
    xls = pd.ExcelFile(file_path)
    for sheet_name in xls.sheet_names:
        df_MC13 = pd.read_excel(xls, sheet_name=sheet_name)
        if not df_MC13.empty:
            print(f"Reading data from sheet: {sheet_name}")
            #print(df_MC6.head(3))
            break
    else:
        print("All sheets are empty in the Excel file.")
else:
    print(f"No file found for date {date_str}: {filename}")


# select particular columns in PG data 
df13 = df_PG[['Material', 'Purchasing Group Details', 'MRP controller details']]
df13.rename(columns={'Material': 'Material Code'}, inplace=True)


merged_df13 = pd.merge(df_MC13, df13, on='Material Code', how='inner')
print(merged_df13.shape)

# Create concatenated column
merged_df13['Material_PG_Concat'] = merged_df13['Material Code'].astype(str) + "_" + merged_df13['Purchasing Group Details'].astype(str)


# Ensure the column is numeric (in case it's read as string)
merged_df13['Stockout Probability (%)'] = pd.to_numeric(merged_df13['Stockout Probability (%)'], errors='coerce')

# Filter rows with stockout probability >= 50%
high_risk_df13 = merged_df13[merged_df13['Stockout Probability (%)'] >= 50]

# Show the filtered DataFrame
print(high_risk_df13.shape)

#print(high_risk_df.head(2))

filtered_df13 = high_risk_df13[high_risk_df13['GIT'] == 0]
print(filtered_df13.shape)

filtered_df13 = filtered_df13[['Running Date','Material Code', 'GIT', 'Purchasing Group Details','Material_PG_Concat']]
filtered_df13.head(7)

In [ ]:
filtered_df12_filtered_df13 = pd.merge(
    filtered_df13,
    filtered_df11_filtered_df12[['Material_PG_Concat']],  # Only use the join column from df_gp
    on='Material_PG_Concat',
    how='inner'
)

print(filtered_df12_filtered_df13.shape)
filtered_df12_filtered_df13.head(7)

In [ ]:
pg_count_13may = (
    filtered_df12_filtered_df13.groupby('Purchasing Group Details')
    .size()
    .reset_index(name='Buyer_counts_13May')
    .sort_values(by=['Purchasing Group Details', 'Buyer_counts_13May'], ascending=[True, False])
)

print(pg_count_13may.shape)
pg_count_13may.head(5)

#14.05.25

In [ ]:
#import os
#from datetime import datetime, timedelta
#import pandas as pd

# Set parameters
num_simulations = 10000  # Adjust as needed

# Generate yesterday's date in 'YYYYMMDD' format
yesterday = datetime.now() - timedelta(days=2)
date_str = yesterday.strftime('%Y%m%d')

# Construct filename dynamically
filename = f"MC_day_wise_{num_simulations}_{date_str}.xlsx"

# Define folder path and file path
folder_path = "/home/VECVNET/zzasom/jupyter/01_Development_work/Black_Forecasting/Deployment/Output"
file_path = os.path.join(folder_path, filename)

# Check and read the file
if os.path.exists(file_path):
    xls = pd.ExcelFile(file_path)
    for sheet_name in xls.sheet_names:
        df_MC14 = pd.read_excel(xls, sheet_name=sheet_name)
        if not df_MC14.empty:
            print(f"Reading data from sheet: {sheet_name}")
            #print(df_MC6.head(3))
            break
    else:
        print("All sheets are empty in the Excel file.")
else:
    print(f"No file found for date {date_str}: {filename}")


# select particular columns in PG data 
df14 = df_PG[['Material', 'Purchasing Group Details', 'MRP controller details']]
df14.rename(columns={'Material': 'Material Code'}, inplace=True)


merged_df14 = pd.merge(df_MC14, df14, on='Material Code', how='inner')
print(merged_df14.shape)

# Create concatenated column
merged_df14['Material_PG_Concat'] = merged_df14['Material Code'].astype(str) + "_" + merged_df14['Purchasing Group Details'].astype(str)


# Ensure the column is numeric (in case it's read as string)
merged_df14['Stockout Probability (%)'] = pd.to_numeric(merged_df14['Stockout Probability (%)'], errors='coerce')

# Filter rows with stockout probability >= 50%
high_risk_df14 = merged_df14[merged_df14['Stockout Probability (%)'] >= 50]

# Show the filtered DataFrame
print(high_risk_df14.shape)

#print(high_risk_df.head(2))

filtered_df14 = high_risk_df14[high_risk_df14['GIT'] == 0]
print(filtered_df14.shape)

filtered_df14 = filtered_df14[['Running Date','Material Code', 'GIT', 'Purchasing Group Details','Material_PG_Concat']]
filtered_df14.head(7)

In [ ]:
filtered_df13_filtered_df14 = pd.merge(
    filtered_df14,
    filtered_df12_filtered_df13[['Material_PG_Concat']],  # Only use the join column from df_gp
    on='Material_PG_Concat',
    how='inner'
)

print(filtered_df13_filtered_df14.shape)
filtered_df13_filtered_df14.head(7)

In [ ]:
pg_count_14may = (
    filtered_df13_filtered_df14.groupby('Purchasing Group Details')
    .size()
    .reset_index(name='Buyer_counts_14May')
    .sort_values(by=['Purchasing Group Details', 'Buyer_counts_14May'], ascending=[True, False])
)

print(pg_count_14may.shape)
pg_count_14may.head(5)

In [ ]:
merged_11_12 = pd.merge(pg_count_11may, pg_count_12may, on='Purchasing Group Details', how='outer')
merged_12_13 = pd.merge(merged_11_12, pg_count_13may, on='Purchasing Group Details', how='outer')
merged_13_14 = pd.merge(merged_12_13, pg_count_14may, on='Purchasing Group Details', how='outer')
#merged_14_15 = pd.merge(merged_13_14, pg_count_15may, on='Purchasing Group Details', how='outer')


# 5. Fill NaN values with 0 and convert to int
merged_13_14[['Buyer_counts_11May', 'Buyer_counts_12May','Buyer_counts_13May','Buyer_counts_14May']] = (
    merged_13_14[['Buyer_counts_11May', 'Buyer_counts_12May','Buyer_counts_13May','Buyer_counts_14May']].fillna(0).astype(int)
)

# 6. Optional display
print(merged_13_14.shape)
#merged_13_14.head(10)


# Assuming your DataFrame is named df
# Example column names: 'Buyer_counts_11May', 'Buyer_counts_15May'

merged_13_14['pending %'] = (merged_13_14['Buyer_counts_14May'] / merged_13_14['Buyer_counts_11May']) * 100
merged_13_14['Efficiency %'] = 100 - merged_13_14['pending %']

# Optional: Round the values for better readability
merged_13_14['pending %'] = merged_13_14['pending %'].round(2).astype(str) + '%'
merged_13_14['Efficiency %'] = merged_13_14['Efficiency %'].round(2).astype(str) + '%'

# Display result
merged_13_14.head(10)

# Week -02

In [ ]:
#import os
#from datetime import datetime, timedelta
#import pandas as pd

# Set parameters
num_simulations = 10000  # Adjust as needed

# Generate yesterday's date in 'YYYYMMDD' format
yesterday = datetime.now() - timedelta(days=4)
date_str = yesterday.strftime('%Y%m%d')

# Construct filename dynamically
filename = f"MC_day_wise_{num_simulations}_{date_str}.xlsx"

# Define folder path and file path
folder_path = "/home/VECVNET/zzasom/jupyter/01_Development_work/Black_Forecasting/Deployment/Output"
file_path = os.path.join(folder_path, filename)

# Check and read the file
if os.path.exists(file_path):
    xls = pd.ExcelFile(file_path)
    for sheet_name in xls.sheet_names:
        df_MC112 = pd.read_excel(xls, sheet_name=sheet_name)
        if not df_MC112.empty:
            print(f"Reading data from sheet: {sheet_name}")
            #print(df_MC6.head(3))
            break
    else:
        print("All sheets are empty in the Excel file.")
else:
    print(f"No file found for date {date_str}: {filename}")


# select particular columns in PG data 
df112 = df_PG[['Material', 'Purchasing Group Details', 'MRP controller details']]
df112.rename(columns={'Material': 'Material Code'}, inplace=True)


merged_df112 = pd.merge(df_MC112, df112, on='Material Code', how='inner')
print(merged_df112.shape)

# Create concatenated column
merged_df112['Material_PG_Concat'] = merged_df112['Material Code'].astype(str) + "_" + merged_df112['Purchasing Group Details'].astype(str)


# Ensure the column is numeric (in case it's read as string)
merged_df112['Stockout Probability (%)'] = pd.to_numeric(merged_df112['Stockout Probability (%)'], errors='coerce')

# Filter rows with stockout probability >= 50%
high_risk_df112 = merged_df112[merged_df112['Stockout Probability (%)'] >= 50]

# Show the filtered DataFrame
print(high_risk_df112.shape)

#print(high_risk_df.head(2))

filtered_df112 = high_risk_df112[high_risk_df112['GIT'] == 0]
print(filtered_df112.shape)

filtered_df112 = filtered_df112[['Running Date','Material Code', 'GIT', 'Purchasing Group Details','Material_PG_Concat']]
filtered_df112.head(7)

merged_dfgp_filtered_df112 = pd.merge(
    filtered_df112,
    df_gp[['Material_PG_Concat']],  # Only use the join column from df_gp
    on='Material_PG_Concat',
    how='inner'
)

print(merged_dfgp_filtered_df112.shape)
merged_dfgp_filtered_df112.head(7)

pg_count_112 = (
    merged_dfgp_filtered_df112.groupby('Purchasing Group Details')
    .size()
    .reset_index(name='Buyer_12May')
    .sort_values(by=['Purchasing Group Details', 'Buyer_12May'], ascending=[True, False])
)

print(pg_count_112.shape)
pg_count_112.head(4)

In [ ]:
filtered_df12_filtered_df13_new = pd.merge(
    filtered_df13,
    merged_dfgp_filtered_df112[['Material_PG_Concat']],  # Only use the join column from df_gp
    on='Material_PG_Concat',
    how='inner'
)

print(filtered_df12_filtered_df13_new.shape)
filtered_df12_filtered_df13_new.head(7)

In [ ]:
pg_count_13 = (
    filtered_df12_filtered_df13_new.groupby('Purchasing Group Details')
    .size()
    .reset_index(name='Buyer_13May')
    .sort_values(by=['Purchasing Group Details', 'Buyer_13May'], ascending=[True, False])
)

print(pg_count_13.shape)
pg_count_13.head(5)

In [ ]:
filtered_df13_filtered_df14_new = pd.merge(
    filtered_df14,
    filtered_df12_filtered_df13_new[['Material_PG_Concat']],  # Only use the join column from df_gp
    on='Material_PG_Concat',
    how='inner'
)

print(filtered_df13_filtered_df14_new.shape)
filtered_df13_filtered_df14_new.head(7)

In [ ]:
pg_count_14 = (
    filtered_df13_filtered_df14_new.groupby('Purchasing Group Details')
    .size()
    .reset_index(name='Buyer_14May')
    .sort_values(by=['Purchasing Group Details', 'Buyer_14May'], ascending=[True, False])
)

print(pg_count_14.shape)
pg_count_14.head(5)

In [ ]:
#import os
#from datetime import datetime, timedelta
#import pandas as pd

# Set parameters
num_simulations = 10000  # Adjust as needed

# Generate yesterday's date in 'YYYYMMDD' format
yesterday = datetime.now() - timedelta(days=1)
date_str = yesterday.strftime('%Y%m%d')

# Construct filename dynamically
filename = f"MC_day_wise_{num_simulations}_{date_str}.xlsx"

# Define folder path and file path
folder_path = "/home/VECVNET/zzasom/jupyter/01_Development_work/Black_Forecasting/Deployment/Output"
file_path = os.path.join(folder_path, filename)

# Check and read the file
if os.path.exists(file_path):
    xls = pd.ExcelFile(file_path)
    for sheet_name in xls.sheet_names:
        df_MC15 = pd.read_excel(xls, sheet_name=sheet_name)
        if not df_MC15.empty:
            print(f"Reading data from sheet: {sheet_name}")
            #print(df_MC15.head(3))
            break
    else:
        print("All sheets are empty in the Excel file.")
else:
    print(f"No file found for date {date_str}: {filename}")


# select particular columns in PG data 
df15 = df_PG[['Material', 'Purchasing Group Details', 'MRP controller details']]
df15.rename(columns={'Material': 'Material Code'}, inplace=True)


merged_df15 = pd.merge(df_MC15, df15, on='Material Code', how='inner')
print(merged_df15.shape)

# Create concatenated column
merged_df15['Material_PG_Concat'] = merged_df15['Material Code'].astype(str) + "_" + merged_df15['Purchasing Group Details'].astype(str)


# Ensure the column is numeric (in case it's read as string)
merged_df15['Stockout Probability (%)'] = pd.to_numeric(merged_df15['Stockout Probability (%)'], errors='coerce')

# Filter rows with stockout probability >= 50%
high_risk_df15 = merged_df15[merged_df15['Stockout Probability (%)'] >= 50]

# Show the filtered DataFrame
print(high_risk_df15.shape)

#print(high_risk_df.head(2))

filtered_df15 = high_risk_df15[high_risk_df15['GIT'] == 0]
print(filtered_df15.shape)

filtered_df15 = filtered_df15[['Running Date','Material Code', 'GIT', 'Purchasing Group Details','Material_PG_Concat']]
filtered_df15.head(7)

In [ ]:
filtered_df14_filtered_df15_new = pd.merge(
    filtered_df15,
    filtered_df13_filtered_df14_new[['Material_PG_Concat']],  # Only use the join column from df_gp
    on='Material_PG_Concat',
    how='inner'
)

print(filtered_df14_filtered_df15_new.shape)
filtered_df14_filtered_df15_new.head(7)

In [ ]:
pg_count_15 = (
    filtered_df14_filtered_df15_new.groupby('Purchasing Group Details')
    .size()
    .reset_index(name='Buyer_15May')
    .sort_values(by=['Purchasing Group Details', 'Buyer_15May'], ascending=[True, False])
)

print(pg_count_15.shape)
pg_count_15.head(5)

In [ ]:
merged1 = pd.merge(pg_count_112, pg_count_13, on='Purchasing Group Details', how='outer')
merged2 = pd.merge(merged1, pg_count_14, on='Purchasing Group Details', how='outer')
merged3 = pd.merge(merged2, pg_count_15, on='Purchasing Group Details', how='outer')
#merged_14_15 = pd.merge(merged_13_14, pg_count_15may, on='Purchasing Group Details', how='outer')


# 5. Fill NaN values with 0 and convert to int
merged3[['Buyer_12May', 'Buyer_13May','Buyer_14May','Buyer_15May']] = (
    merged3[['Buyer_12May', 'Buyer_13May','Buyer_14May','Buyer_15May']].fillna(0).astype(int)
)

# 6. Optional display
print(merged3.shape)
#merged_13_14.head(10)


# Assuming your DataFrame is named df
# Example column names: 'Buyer_counts_11May', 'Buyer_counts_15May'

merged3['pending %'] = (merged3['Buyer_15May'] / merged3['Buyer_12May']) * 100
merged3['Efficiency %'] = 100 - merged3['pending %']

# Optional: Round the values for better readability
merged3['pending %'] = merged3['pending %'].round(2).astype(str) + '%'
merged3['Efficiency %'] = merged3['Efficiency %'].round(2).astype(str) + '%'

# Display result
merged3.head(10)

In [ ]:
import pandas as pd


# Merge on 'Purchasing Group Details' column
combined_df = pd.merge(merged_13_14, merged3, on='Purchasing Group Details', suffixes=('_11to14', '_12to15'))
print(combined_df.shape)
# Show result
combined_df.head(10)

In [ ]:
combined_df.to_excel('BLACK_FORECATING_DASHBOARD_OUTPUt.xlsx', index=False)

print("Excel file 'BLACK_FORECATING_DASHBOARD_OUTPUt.xlsx' has been saved.")

#### code for loop creation 

In [9]:
import os
import pandas as pd
from datetime import datetime, timedelta
from functools import reduce

# === Constants ===
NUM_SIMULATIONS = 10000
INPUT_FOLDER = "/home/VECVNET/zzasom/jupyter/01_Development_work/Black_Forecasting/Deployment/Input"
OUTPUT_FOLDER = "/home/VECVNET/zzasom/jupyter/01_Development_work/Black_Forecasting/Deployment/Output"
BASE_DATE = datetime(2025, 5, 26)  # Fixed base date: 12 May 2025
DAYS_AHEAD = 3  # Today + next 3 days

# === Load PG Mapping File ===
pg_file_path = os.path.join(INPUT_FOLDER, "MRPcontrollerPurchasingGrp.csv")
if not os.path.exists(pg_file_path):
    raise FileNotFoundError("PG file not found.")

df_pg = pd.read_csv(pg_file_path)
df_pg['Material_PG_Concat'] = df_pg['Material'].astype(str) + "_" + df_pg['Purchasing Group Details'].astype(str)
df_pg_merge = df_pg[['Material', 'Purchasing Group Details', 'MRP controller details']].rename(
    columns={'Material': 'Material Code'}
)

# === Function to Process a Single Date ===
def process_day(date_obj, filter_from_df):
    date_str = date_obj.strftime('%Y%m%d')
    file_name = f"MC_day_wise_{NUM_SIMULATIONS}_{date_str}.xlsx"
    file_path = os.path.join(OUTPUT_FOLDER, file_name)

    if not os.path.exists(file_path):
        print(f"[{date_str}] File not found: {file_name}")
        return None, None

    try:
        xls = pd.ExcelFile(file_path)
        for sheet_name in xls.sheet_names:
            df_mc = pd.read_excel(xls, sheet_name=sheet_name)
            if not df_mc.empty:
                break
        else:
            print(f"[{date_str}] All sheets are empty.")
            return None, None

        # Merge PG info
        df_mc = pd.merge(df_mc, df_pg_merge, on='Material Code', how='inner')
        df_mc['Material_PG_Concat'] = df_mc['Material Code'].astype(str) + "_" + df_mc['Purchasing Group Details'].astype(str)
        df_mc['Stockout Probability (%)'] = pd.to_numeric(df_mc['Stockout Probability (%)'], errors='coerce')

        # Filter
        high_risk_df = df_mc[(df_mc['Stockout Probability (%)'] >= 50) & (df_mc['GIT'] == 0)]
        filtered_df = high_risk_df[['Running Date', 'Material Code', 'GIT', 'Purchasing Group Details', 'Material_PG_Concat']]

        if filter_from_df is not None:
            filtered_df = filtered_df[filtered_df['Material_PG_Concat'].isin(filter_from_df['Material_PG_Concat'])]

        # Count high-risk parts per PG
        col_label = f"Buyer_counts_{date_obj.strftime('%d%b')}"
        pg_counts = (
            filtered_df.groupby('Purchasing Group Details')
            .size()
            .reset_index(name=col_label)
        )

        return filtered_df, pg_counts

    except Exception as e:
        print(f"[{date_str}] Error processing: {e}")
        return None, None

# === Process Multiple Days ===
filtered_df_acc = None
pg_counts_all = []

for i in range(DAYS_AHEAD + 1):  # today to today + 3
    run_date = BASE_DATE + timedelta(days=i)
    filtered_df_acc, pg_count = process_day(run_date, filtered_df_acc)
    if pg_count is not None:
        pg_counts_all.append(pg_count)

# === Merge and Compute Efficiency ===
if pg_counts_all:
    merged_pg_counts = reduce(
        lambda left, right: pd.merge(left, right, on='Purchasing Group Details', how='outer'),
        pg_counts_all
    )

    # Fill NaN with 0 and convert to int
    count_cols = [col for col in merged_pg_counts.columns if col.startswith("Buyer_counts_")]
    merged_pg_counts[count_cols] = merged_pg_counts[count_cols].fillna(0).astype(int)

    if len(count_cols) >= 2:
        first_col, last_col = count_cols[0], count_cols[-1]
        merged_pg_counts['pending %'] = (merged_pg_counts[last_col] / merged_pg_counts[first_col].replace(0, 1)) * 100
        merged_pg_counts['Efficiency %'] = 100 - merged_pg_counts['pending %']

        merged_pg_counts['pending %'] = merged_pg_counts['pending %'].round(2).astype(str) + '%'
        merged_pg_counts['Efficiency %'] = merged_pg_counts['Efficiency %'].round(2).astype(str) + '%'

    print(merged_pg_counts.head(2))
else:
    print("No data processed.")


  Purchasing Group Details  Buyer_counts_26May  Buyer_counts_27May  \
0               Akshay Raj                  35                  27   
1           Amarjeet Yadav                  37                  28   

   Buyer_counts_28May  Buyer_counts_29May pending % Efficiency %  
0                  20                  12    34.29%       65.71%  
1                  15                  13    35.14%       64.86%  


In [10]:
merged_pg_counts.head(7)

,Purchasing Group Details,Buyer_counts_26May,Buyer_counts_27May,Buyer_counts_28May,Buyer_counts_29May,pending %,Efficiency %
0,Akshay Raj,35,27,20,12,34.29%,65.71%
1,Amarjeet Yadav,37,28,15,13,35.14%,64.86%
2,Arun Kumar Shah,38,24,15,11,28.95%,71.05%
3,Ashish Purohit,12,9,7,7,58.33%,41.67%
4,Deepak Deshpande,32,29,28,26,81.25%,18.75%
5,Deepak Raipure,48,32,21,13,27.08%,72.92%
6,Girish R Sonawane,52,31,21,16,30.77%,69.23%


In [11]:
merged_pg_counts.to_excel('BF_26May_Dashboard_file.xlsx', index=False)


In [12]:
# Convert Pandas DataFrame to PySpark DataFrame
spark_df = spark.createDataFrame(merged_pg_counts)


In [13]:
# Write PySpark DataFrame to the database
spark_df.write.mode("overwrite").format("jdbc").options(
    driver=driver,
    user=user,
    password=password,
    url=url,
    dbtable="Black_Forecasting_Dashboard"
).save()

In [ ]:
print("final output store in the table")

In [14]:
spark.stop()